In [1]:
from keras.models import load_model
import keraOverlay as overlay
import numpy as np
import matplotlib.pyplot as plt 
import utils

/home/yuan/Software/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load preliminary trained model
model = load_model('Scale_down64.h5', custom_objects={'dice_coef_loss': overlay.dice_coef_loss,
                                                      'dice_coef': overlay.dice_coef})

In [3]:
# Load data and masks as dicom files
data_dir = '../../data'                   #Please change the path accordingly before make any changes
normalNames = data_dir + '/testdata.txt'
img_dir = data_dir + '/256Auto/images'
msk_dir = data_dir + '/256Auto/masks'

In [4]:
#Load data to check first as lists
#from imp import reload
#reload(utils)
x_list, y_list = utils.load_data(img_dir, msk_dir, data_name_list, padding=True, sigma_image=sigma, scaling=4, OPaslist=True)

NameError: name 'data_name_list' is not defined

In [ ]:
#predict all data and calculate dice score and store
from keras import backend
n_slice = 64
size = 64
print(len(x_list))
index = 9
confidence = 0.9

predictions = []
dice_scores = []
text_file = open("dice_scores_lowres.txt", "w")
text_file2 = open("dice_scores_up.txt", "w")   #also try to scale back the image and calculate the dice

for i, image in enumerate(x_list):
    mask = y_list[i]
    predictions.append(model.predict(image.reshape(1, n_slice, size, size, 1), steps=1))
    predictions[i][predictions[i] >= confidence] = 1
    predictions[i][predictions[i] < confidence] = 1

    dice_scores.append(overlay.dice_coef(mask[np.newaxis, ...], predictions[i]))


#plt.hist(prediction.flatten())
for i in dice_scores:
    text_file.write(backend.eval(i) + "\n")

text_file.close()



utils.plot_3d(np.squeeze(predictions[index]), y_list[index], threshold=0.5, threshold2=0.5)